In [106]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
from tensorflow import lite as tflite
import time

import h5py
import os
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Convolution1D, MaxPooling2D, Convolution2D, DepthwiseConv2D
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import backend as K
from tensorflow.keras.activations import relu, softmax

from keras.callbacks import ModelCheckpoint
from keras.callbacks import RemoteMonitor
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import f1_score


In [2]:
wav_root = '../data/audio_1sec/'

In [17]:
# labels = pd.read_csv('../labels/audio_1sec.csv',  header=None, names=["path", "yes", "no", "not_sure","subject_set"])
# labels['path'] = labels['path'].astype(str) + '.wav'

# # calculate the percentage of "yes" responses for each label
# labels['calc'] = (labels['yes'].astype(int)*1 + labels['not_sure'].astype(int)*0.5) / (labels['yes'] + labels['no'] + labels['not_sure'])

# # filter the labels to get only the ones with a definite answer
# filtered_labels = labels[(labels['yes'] + labels['no']) > 0]

# # sort the filtered labels by the calculated percentage in descending order
# sorted_labels = filtered_labels.sort_values(by='calc', ascending=False)

# # select the highest 5000 labels where calc > 0.5
# num_high = min(5000, len(sorted_labels[sorted_labels['calc'] > 0.5]))
# selected_high = sorted_labels[:num_high]

# # select the lowest 5000 labels where calc < 0.5
# num_low = min(10000, len(sorted_labels[sorted_labels['calc'] < 0.5]))
# selected_low = sorted_labels[-num_low:]

# # concatenate the selected high and low labels and shuffle them randomly
# labels = pd.concat([selected_high, selected_low], axis=0).sample(frac=1).reset_index(drop=True)

# # calculate the binary labels (1 for "yes", 0 for "no")
# labels['res'] = (labels['yes'].astype(int)*1 + labels['not_sure'].astype(int)*0.5) / (labels['yes'] + labels['no'] + labels['not_sure']) >= 0.5

# # create y from the selected labels
# n = len(labels)
# y = np.zeros((n, 2))
# y[:,1] = np.array(labels['res']).astype(int)
# y[:,0] = 1 - y[:,1]

# feature_type = 'mfcc'


In [33]:
labels = pd.read_csv('../labels/audio_1sec.csv',  header=None, names=["path", "yes", "no", "not_sure","subject_set"])
labels['path'] = labels['path'].astype(str) + '.wav'
labels['calc'] = (labels['yes'].astype(int)*1 + labels['not_sure'].astype(int)*0.5) / (labels['yes'] + labels['no'] + labels['not_sure']) 



labels['res'] = (labels['yes'].astype(int)*1 + labels['not_sure'].astype(int)*0.5) / (labels['yes'] + labels['no'] + labels['not_sure']) >= 0.5


n = len(labels.index.values.tolist())

y = np.zeros((n, 2))

y[:,1] = np.array(np.array(labels['res']).astype(int)).astype(int)
y[:,0] = 1-y[:,1].astype(int)

feature_type = 'mfcc'


In [34]:
# # Set window length in seconds
# win_len = 0.1

# # Set FFT size based on window length and sampling rate
# nfft = int(win_len * 8000)

# # Set number of MFCCs per frame
# n_mfcc = 40

# # Load list of audio file paths
# wav_path = wav_root + labels["path"]
# file_list = list(wav_path)

# # Initialize feature matrix
# spec_matrix = np.zeros((len(file_list)*49, n_mfcc))

# # Extract MFCC features from each file and store in feature matrix
# for i, file_path in enumerate(file_list):
#     wav, fs = librosa.load(file_path, sr=None)
#     mfccs = librosa.feature.mfcc(y=wav, sr=fs, n_mfcc=n_mfcc, n_fft=nfft*4, hop_length=nfft)
    
#     # Pad or truncate the feature matrix to have shape (49, n_mfcc)
#     if mfccs.shape[1] < 49:
#         mfccs = np.pad(mfccs, ((0, 0), (0, 49 - mfccs.shape[1])), mode='constant')
#     elif mfccs.shape[1] > 49:
#         mfccs = mfccs[:, :49]

#     # Store MFCC features in the spec_matrix
#     spec_matrix[i*49:(i+1)*49,:] = mfccs.T

In [35]:
win_len = 0.1

nfft = int(win_len * 8000)
wav_path = wav_root + labels["path"]



wav, fs = librosa.load(wav_path.iloc[0],sr=None)  
spec = librosa.feature.mfcc(y=wav, sr=fs, n_mfcc=13, n_fft=nfft*4, hop_length=nfft)    
l_spec = np.shape(spec)[1]  
h_spec = np.shape(spec)[0]  
spec_matrix = np.zeros([len(y), h_spec, l_spec])

start_time = time.time()
for i in range(0, n):
#     if count[i] == 1:
        
        # Create X matrix
    audio_file, fs = librosa.load(wav_path.loc[i],sr=None)
    spec_matrix[i] = librosa.feature.mfcc(y=audio_file,sr=fs, n_mfcc=13, n_fft=nfft*4, hop_length=nfft)            
        
    if i%1000 == 0:
        time_used = time.time()-start_time
        time_total = time_used * n / (i+1)
        
        print('Iteration', str(i), "%i"%(time_used), "%i"%(time_total))

Iteration 0 0 515
Iteration 1000 11 907
Iteration 2000 21 846
Iteration 3000 31 831
Iteration 4000 40 804
Iteration 5000 48 773
Iteration 6000 56 755
Iteration 7000 65 747
Iteration 8000 78 780
Iteration 9000 89 795
Iteration 10000 100 802
Iteration 11000 111 805
Iteration 12000 122 815
Iteration 13000 133 819
Iteration 14000 144 819
Iteration 15000 154 821
Iteration 16000 166 827
Iteration 17000 177 830
Iteration 18000 188 833
Iteration 19000 199 835
Iteration 20000 210 836
Iteration 21000 220 837
Iteration 22000 231 837
Iteration 23000 241 836
Iteration 24000 251 834
Iteration 25000 261 831
Iteration 26000 271 832
Iteration 27000 282 832
Iteration 28000 292 831
Iteration 29000 302 831
Iteration 30000 313 832
Iteration 31000 323 831
Iteration 32000 334 832
Iteration 33000 345 832
Iteration 34000 355 832
Iteration 35000 365 832
Iteration 36000 376 832
Iteration 37000 386 832
Iteration 38000 397 833
Iteration 39000 408 834
Iteration 40000 419 835
Iteration 41000 429 835
Iteration 42000 

In [36]:
print(len(spec_matrix))
print(len(wav_path))

79662
79662


In [37]:
file_names = ['data_'+feature_type+'not_sure_single_into_0_5.h5',
              'label_'+feature_type+'not_sure_single_into_0_5.h5']

for file_name in file_names:
    if not os.path.isfile('../proc_data/' + file_name):
        hf = h5py.File('../proc_data/data_' + feature_type +
                       'not_sure_single_into_0_5.h5', 'w')
        hf.create_dataset('../proc_data/data_' + feature_type + '_majority_labels_not_sure_single_into_0_5',
                          data=spec_matrix)
        hf.close()

        hf = h5py.File('../proc_data/label_' + feature_type +
                       'not_sure_single_into_0_5.h5', 'w')
        hf.create_dataset('../proc_data/label_' + feature_type +
                          '_majority_labels_not_sure_single_into_0_5', data=y)
        hf.close()


In [38]:
save_name = 'not_sure_single_into_0_5'
hf = h5py.File('../proc_data/data_' + feature_type + save_name + '.h5', 'r')
spec_matrix_read = np.array(hf.get('../proc_data/data_' + feature_type + '_majority_labels_' + save_name))
hf.close()
hf = h5py.File('../proc_data/label_' + feature_type + save_name + '.h5', 'r')
y = np.array(hf.get('../proc_data/label_' + feature_type + '_majority_labels_' + save_name))
hf.close()

In [39]:
spec_matrix_db = np.zeros_like(spec_matrix_read)
for i, spec in enumerate(spec_matrix_read):
    spec_matrix_db[i] = librosa.power_to_db(spec,ref=np.max)

In [87]:
    def get_model3():
        X_train, X_test, y_train, y_test = train_test_split(spec_matrix_db, np.array(y), test_size=0.33,
                                                            random_state=45)
        # Normalise by statistics of training data
        X_train_norm = (X_train - np.mean(X_train))/np.std(X_train)
        X_test_norm = (X_test - np.mean(X_train))/np.std(X_train)
        X_train_tf = X_train_norm.reshape(X_train_norm.shape[0], 1, X_train_norm.shape[1], X_train_norm.shape[2])
        X_test_tf = X_test_norm.reshape(X_test_norm.shape[0], 1, X_test_norm.shape[1], X_test_norm.shape[2])
     
        ################################ CONVOLUTIONAL NEURAL NETWORK ################################
        ## NN parameters
        class_weight = {0: 1.,
                        1: 10.,
                        }
        input_shape = (1, X_train_tf.shape[2], X_train_tf.shape[-1])
        print(input_shape)
     
        model = Sequential()
        n_dense = 128
        nb_classes = 2
        # number of convolutional filters
        nb_conv_filters = 32
        # num_hidden = 236
        nb_conv_filters_2 = 64
        convout1 = Activation('relu')
        convout2 = Activation('relu')
     
        model.add(Conv2D(nb_conv_filters, kernel_size = (3,3),
             activation = 'relu', padding = 'same', strides = 1,
             input_shape = input_shape))
        model.add(MaxPooling2D(pool_size=(1, 2), strides=(1, 2)))
        model.add(Conv2D(nb_conv_filters_2, kernel_size = (3,3),
             activation = 'relu', padding = 'same'))
        model.add(MaxPooling2D(pool_size=(1, 2), strides=(1, 2)))
        # model.add(Conv2D(nb_conv_filters_2, kernel_size = (5,5),
        #      activation = 'relu', padding = 'valid'))
        # model.add(MaxPooling2D(pool_size=(2, 2)))
     
        model.add(Dropout(0.2))  
        model.add(Flatten())
        # Shared between MLP and CNN:
        model.add(Dense(n_dense, activation='relu'))
        model.add(Dense(nb_classes, activation='softmax'))
        model.compile(loss='categorical_crossentropy',
                        optimizer='adadelta',
                        metrics=['accuracy'])
     
        model.fit(x=X_train_tf, y=y_train, batch_size=None, epochs=100, verbose=1, callbacks=None, validation_split=0.0,
                  validation_data=None,
                  shuffle=True, class_weight=class_weight, sample_weight=None, initial_epoch=0,
                  steps_per_epoch=None, validation_steps=None)
     
        
        loss, acc = model.evaluate(x=X_test_tf, y=y_test, batch_size=None, verbose=0, sample_weight=None, steps=None)
        print('Test loss:', loss)
        print('Test accuracy:', acc)
        
        return model,np.mean(X_train),np.std(X_train)

In [88]:
def get_model():
    X_train, X_test, y_train, y_test = train_test_split(spec_matrix_db, y, test_size=0.33, random_state=45)
    
    X_train_norm = (X_train - np.mean(X_train)) / np.std(X_train)
    X_test_norm = (X_test - np.mean(X_train)) / np.std(X_train)

    # number of files, number of time steps , number of features
    X_train_tf = X_train_norm.reshape(
        X_train_norm.shape[0], X_train_norm.shape[1], X_train_norm.shape[2])
    X_test_tf = X_test_norm.reshape(
        X_test_norm.shape[0], X_test_norm.shape[1], X_test_norm.shape[2])

    
    model = Sequential()
    
    model.add(Conv1D(32, kernel_size=3, activation='relu',
              input_shape=(X_train_tf.shape[1], X_train_tf.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

    model.fit(X_train_tf, y_train, batch_size=None, epochs=3,
              verbose=1, class_weight={0: 1., 1: 10.})
    
    loss, acc = model.evaluate(X_test_tf, y_test, batch_size=None, verbose=0)
    print('Test loss:', loss)
    print('Test accuracy:', acc)
    

    
    return model,np.mean(X_train),np.std(X_train)

In [89]:
def get_model2():
    X_train, X_test, y_train, y_test = train_test_split(
        spec_matrix_db, y, test_size=0.33, random_state=45)

    X_train_norm = (X_train - np.mean(X_train)) / np.std(X_train)
    X_test_norm = (X_test - np.mean(X_train)) / np.std(X_train)

    # reshape to add the channels dimension
    X_train_norm = np.expand_dims(X_train_norm, axis=3)
    X_test_norm = np.expand_dims(X_test_norm, axis=3)

    model = Sequential()

    model.add(Conv2D(filters=8, kernel_size=(8, 10), strides=(2, 2), activation='relu',
              input_shape=(X_train_norm.shape[1], X_train_norm.shape[2], X_train_norm.shape[3])))
    model.add(Flatten())
    model.add(Dense(3, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

    model.fit(X_train_norm, y_train, batch_size=None, epochs=10,
              verbose=1, class_weight={0: 1., 1: 10.})

    loss, acc = model.evaluate(X_test_norm, y_test, batch_size=None, verbose=0)
    print('Test loss:', loss)
    print('Test accuracy:', acc)

    return model, np.mean(X_train), np.std(X_train)


In [90]:
model,mean,std = get_model3()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


(1, 13, 11)
Epoch 1/100
1668/1668 [==============================] - 6s 3ms/step - loss: 1.3679 - accuracy: 0.7791
Epoch 2/100
1668/1668 [==============================] - 4s 3ms/step - loss: 1.3418 - accuracy: 0.5806
Epoch 3/100
1668/1668 [==============================] - 4s 3ms/step - loss: 1.3258 - accuracy: 0.4363
Epoch 4/100
1668/1668 [==============================] - 5s 3ms/step - loss: 1.3131 - accuracy: 0.3976
Epoch 5/100
1668/1668 [==============================] - 5s 3ms/step - loss: 1.3015 - accuracy: 0.4074
Epoch 6/100
1668/1668 [==============================] - 5s 3ms/step - loss: 1.2900 - accuracy: 0.4296
Epoch 7/100
1668/1668 [==============================] - 5s 3ms/step - loss: 1.2783 - accuracy: 0.4572
Epoch 8/100
1668/1668 [==============================] - 4s 3ms/step - loss: 1.2697 - accuracy: 0.4958
Epoch 9/100
1668/1668 [==============================] - 5s 3ms/step - loss: 1.2601 - accuracy: 0.5265
Epoch 10/100
1668/1668 [==============================] - 5s 

2023-04-24 12:54:14.159326: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,3,64]
	 [[{{node inputs}}]]
2023-04-24 12:54:14.354321: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,3,64]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /tmp/tmpfkn7jfpg/assets


INFO:tensorflow:Assets written to: /tmp/tmpfkn7jfpg/assets
2023-04-24 12:54:15.844765: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-04-24 12:54:15.844832: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-04-24 12:54:15.845192: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpfkn7jfpg
2023-04-24 12:54:15.849485: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-04-24 12:54:15.849519: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpfkn7jfpg
2023-04-24 12:54:15.859674: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-04-24 12:54:15.919175: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmpfkn7jfpg
2023-04-24 12:54:15.936034: I tensorflow/cc/saved_model/loader.cc:314] SavedModel

In [91]:
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [92]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('mean.pkl', 'wb') as f:
    pickle.dump(mean, f)

with open('std.pkl', 'wb') as f:
    pickle.dump(std, f)


In [93]:
import pickle

with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('mean.pkl', 'rb') as f:
    mean = pickle.load(f)

with open('std.pkl', 'rb') as f:
    std = pickle.load(f)

In [94]:
wav_root = '../data/audio_1sec/'

labels = pd.read_csv('../labels/audio_1sec.csv',  header=None, names=["path", "yes", "no", "not_sure","subject_set"])
labels['path'] = labels['path'].astype(str) + '.wav'

labels['res'] = (labels['yes'].astype(int)*1 + labels['not_sure'].astype(int)*0.5) / (labels['yes'] + labels['no'] + labels['not_sure']) >= 0.5
n = len(labels.index.values.tolist())

y = np.zeros((n, 2))

y[:,1] = np.array(np.array(labels['res']).astype(int)).astype(int)
y[:,0] = 1-y[:,1].astype(int)

In [105]:
import IPython.display as ipd
import random

# Load the audio file you want to predict
wav_root = '../data/audio_1sec/'
wav_path = wav_root + labels["path"]

for i in range(10):
    index = random.randint(0,len(wav_path)-1)
    wav_file = wav_path[index]
    wav, sr = librosa.load(wav_file, sr=None)

    # Extract the features from the audio file
    nfft = int(win_len * sr)
    spec = librosa.feature.mfcc(y=wav, sr=sr, n_mfcc=13, n_fft=nfft*4, hop_length=nfft)
    spec_norm = (spec - mean) / std

    # Reshape the data to match the input shape of the model
    spec_norm_reshaped = spec_norm.reshape(1, 1, spec_norm.shape[0], spec_norm.shape[1])

    # Make a prediction
    pred = model.predict(spec_norm_reshaped)

    # Print the predicted class
    if pred.argmax(axis=1) == 0:
        print('The audio file', wav_file, 'is of class 0', ', Label ', (labels.iloc[index]['yes'].astype(int)*1 + labels.iloc[index]['not_sure'].astype(int)*0.5) / (labels.iloc[index]['yes'] + labels.iloc[index]['no'] + labels.iloc[index]['not_sure']) >= 0.5)
    else:
        print('The audio file', wav_file, 'is of class 1', ', Label ', (labels.iloc[index]['yes'].astype(int)*1 + labels.iloc[index]['not_sure'].astype(int)*0.5) / (labels.iloc[index]['yes'] + labels.iloc[index]['no'] + labels.iloc[index]['not_sure']) >= 0.5)
    
    # Display the playable audio file
    ipd.display(ipd.Audio(wav, rate=sr))


1/1 [==============================] - 0s 89ms/step
The audio file ../data/audio_1sec/604483533.wav is of class 0 , Label  False


1/1 [==============================] - 0s 40ms/step
The audio file ../data/audio_1sec/89870006.wav is of class 0 , Label  False


1/1 [==============================] - 0s 69ms/step
The audio file ../data/audio_1sec/604430002.wav is of class 0 , Label  False


1/1 [==============================] - 0s 112ms/step
The audio file ../data/audio_1sec/77280005.wav is of class 0 , Label  False


1/1 [==============================] - 0s 116ms/step
The audio file ../data/audio_1sec/604260021.wav is of class 1 , Label  False


1/1 [==============================] - 0s 117ms/step
The audio file ../data/audio_1sec/48050006.wav is of class 0 , Label  False


1/1 [==============================] - 0s 132ms/step
The audio file ../data/audio_1sec/604493015.wav is of class 0 , Label  False


1/1 [==============================] - 0s 61ms/step
The audio file ../data/audio_1sec/604470597.wav is of class 0 , Label  False


1/1 [==============================] - 0s 34ms/step
The audio file ../data/audio_1sec/90440028.wav is of class 0 , Label  False


1/1 [==============================] - 0s 35ms/step
The audio file ../data/audio_1sec/76310037.wav is of class 0 , Label  False


In [15]:
import serial

# replace 'COM4' with the serial port where your Arduino is connected
arduino = serial.Serial('/dev/ttyACM0', 9600)

while True:
    user_input = input("Enter 1 to turn on LED or 0 to turn off LED: ")
    arduino.write(user_input.encode())


SerialException: [Errno 13] could not open port /dev/ttyACM0: [Errno 13] Permission denied: '/dev/ttyACM0'

AttributeError: module 'numba' has no attribute 'core'